In [ ]:
import os

import papermill as pm

In [ ]:
data_path = "../../data/papermill/process_data"
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

In [ ]:
def run_notebook(script):
    pm.execute_notebook(
        script,
        os.path.join(data_path, script),
        parameters=dict(),
    )

In [ ]:
run_notebook("ProcessOfflineDatabase.ipynb")

In [ ]:
run_notebook("ProcessAnimeLists.ipynb")

In [ ]:
run_notebook("PruneAnimeLists.ipynb")

In [ ]:
run_notebook("RelabelIds.ipynb")

In [ ]:
run_notebook("RelatedAnimeGraph.ipynb")

In [ ]:
run_notebook("GenerateSplits.ipynb")

In [ ]:
run_notebook("CompressSplits.ipynb")

In [ ]:
print("Success!")